In [1]:
number_of_shots = 3

In [2]:
experiment_name = "2024.05.07-04_Mixtral_7B_instruct_API_3-few-shots_SEMANTIC_SIMILARITY"

In [3]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [4]:
!pip install -q openai #--progress-bar off

In [5]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [6]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [7]:
# output_datapath = "data/output/2023.11.16-1_Mistral_7B_instruct_API/"
# final_report_name = 'report_sdg_classification_Mistral_7B.xlsx'
output_datapath = f"data/output/{experiment_name}/"
log_filename = f"logs/{experiment_name}.log"

final_report_name = 'report_sdg_classification_Mistral_7B.xlsx'

In [8]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [9]:
top7_train_data_sim_to_each_test_data_filename = "../data/input/top7_train_data_sim_to_each_test_data.pickle"

with open(top7_train_data_sim_to_each_test_data_filename, 'rb') as f:
    top7_train_data_sim_to_each_test_data = pickle.load(f)

In [10]:
data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx")

In [11]:
# few_shots = ""

# for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
#     input_text = "Input text: \""+row['text']+"\"\n"
#     output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'
    
#     few_shots += input_text
#     few_shots += output_text
    
# print(few_shots)

In [12]:
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx")
data = data.drop(columns=['Unnamed: 0'])
data.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125


In [13]:
# template_part1 = """[INST]
# <<SYS>>
# In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The text to be processed is:
# <</SYS>>

# '''
# """

# template_part2 = """
# '''

# <<SYS>>
# Answer with the label only using the format:---LABEL---.
# <</SYS>>
# [/INST]
# """

# system_prompt = f"""In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The following are {number_of_shots} examples of expected classification given an input text:

# """+few_shots+"""

# Answer with the label only using the format:---LABEL---
# """


In [14]:
data_sel = data #.iloc[:5]

In [15]:
# Sposta il prompt dentro il ciclo

In [16]:
import time

# cnt = 0
results = []
for index, row in data_sel.iterrows():
    
    cnt = str(index)
    
    file = open(log_filename, 'a+', encoding='utf-8')
    file.write("\n\n##################################################\n")
    
    text_id = row['text_id']
    text = row["text"]
    sdg_true = row["sdg"]

    few_shots_list = top7_train_data_sim_to_each_test_data[row['text_id']][:number_of_shots]    
    
    few_shots = ""

    for train_data_id in few_shots_list:

        # print("##################################################")
        # print(train_data_id)

        train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]

        # print(train_row)

        # print(train_row.text.values[0])
        # print(train_row.sdg.values[0])

        input_text = "Input text: \""+train_row.text.values[0].replace('\n','')+"\"\n"
        output_text = "Output: \"---SDG "+str(train_row.sdg.values[0])+'---\"\n\n'

        few_shots += input_text
        few_shots += output_text    
    
    system_prompt = f"""In the following lines, a text enclosed in triple quotes will be given to you.

    The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

    The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

    If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "---SDG 0---".

    The following are {number_of_shots} examples of expected classification given an input text:

    """+few_shots+"""

    Answer with the label only using the format:---LABEL---
    """    
    
    #query = template_part1 + text + template_part2
    query = text
    
    try:
        body = {
          "model": model_name,
          "messages": [
              {"role": "system", "content": system_prompt}, 
              {"role": "user", "content": query}],
          "temperature": 0.0
        }

        result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
        print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
        file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+str(result)+"\n")
    except Exception as e:
        result = str(e)
        print(cnt, text_id, sdg_true, "ERROR: "+result)
        file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+" "+"ERROR: "+result+"\n")        
    
#     print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
    
    result_extended = (cnt, text_id, sdg_true, result)
    #print(result_extended)
    
    with open(output_datapath+f'sdg_classification_{model_name.replace("/","_")}_{cnt}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)
    
    results.append(result_extended)
    #cnt+=1
    
    file.close()
    
    time.sleep(0.5)
    
    

0 6668 0  ---SDG 11--- (Sustainable Cities and Communities) as the text refers to the restoration of service and lack of injuries in a subway derailment, which can be related to the goal of making cities and human settlements inclusive, safe, resilient and sustainable.
1 6099 0  ---SDG 0---

The text is about a model's photo shoot for the Sports Illustrated Swimsuit issue in the Maldives. It does not contain any information related to the Sustainable Development Goals.
2 6471 0  ---SDG 16---

The topic of this text is the suspension of a life sentence awarded to a convict in a criminal case in India, and the lack of evidence to support certain claims made against the convict. This is related to SDG 16, which aims to promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels. The text discusses the legal process and the administration of justice, which are key component

41 6772 0  ---SDG 0---
42 6340 0  ---SDG 0---

The text provided discusses the second season of the television show "Star Trek: Strange New Worlds" and its recent episode, which features a courtroom drama. The text also includes comments from cast members about their favorite episodes. However, this text does not provide any information that would allow it to be classified under any of the Sustainable Development Goals (SDGs) adopted by the United Nations. Therefore, the appropriate classification for this text is "---SDG 0---".
43 4505 0  ---SDG 3---
44 1776 0  ---SDG 0---
45 1529 0  ---SDG 9---
46 1838 0  ---SDG 0---

The text provided is about a football transfer news and does not directly address any of the Sustainable Development Goals (SDGs). Therefore, it is appropriate to classify it under "---SDG 0---".
47 1427 0  ---SDG 15---

Explanation:
The text focuses on species diversity and the importance of conserving biological diversity and sustainably using resources, which aligns 

78 5968 1  ---SDG 1---
79 2815 1  ---SDG 1---

The text discusses the issue of child poverty and its relation to the provision of basic needs such as drinking water and household waste treatment methods in several countries. This aligns with SDG 1, which aims to end poverty in all its forms everywhere.
80 3635 1  ---SDG 1---
81 2391 1  ---SDG 1---

The text discusses the impact of economic policies on poverty reduction, specifically focusing on the effect of economic growth and income redistribution on poverty rates. This aligns with the goals of Sustainable Development Goal (SDG) 1, which aims to end poverty in all its forms everywhere. The text emphasizes the importance of considering the distribution of income and the severity of poverty, which are key aspects of SDG 1's focus on poverty eradication.
82 841 1  ---SDG 1---
83 1373 1  ---SDG 1--- (No Poverty) is the most appropriate classification for this text. The text discusses the concept of poverty, its measurement, and its effec

119 5693 1  ---SDG 5---
120 2068 2  ---SDG 2---
121 6274 2  ---SDG 2---
122 4363 2  ---SDG 2---
123 6255 2  ---SDG 5---

The text discusses the challenges faced by women in rural areas, particularly in relation to their multiple responsibilities and limited access to resources. It highlights the need for greater participation by and capacity development of women farmers, as well as the importance of addressing gender discrimination in land distribution practices and property rights. These issues are closely aligned with SDG 5, which aims to achieve gender equality and empower all women and girls.
124 3445 2  ---SDG 2---
125 5756 2  ---SDG 2---
126 720 2  ---SDG 2---
127 6780 2  ---SDG 2---
128 2407 2  ---SDG 12--- (Sustainable consumption and production)
129 3672 2  ---SDG 2---

Explanation:

The text given in the example discusses food security and its various dimensions, including availability, access, and utilization of food. It also mentions the importance of agricultural productiv

164 4120 2  ---SDG 2--- (The issues discussed in the text, such as food security and rising food prices, align with the focus of SDG 2, which is to end hunger, achieve food security and improved nutrition, and promote sustainable agriculture.)
165 894 2  ---SDG 2---

The text discusses the importance of improving local food production, particularly in developing countries, in order to address food insecurity. It highlights the risks of desertification and population shifts in rural areas, and the potential benefits of policy interventions to improve the attractiveness of agriculture and rural livelihoods. This focus on reducing hunger and improving food security aligns with SDG 2.
166 1114 2  ---SDG 2---
167 211 2  ---SDG 2---

The text discusses issues related to agricultural production, specifically the use of fertiliser subsidies. SDG 2 aims to "end hunger, achieve food security and improved nutrition and promote sustainable agriculture." The text highlights the need for cost-effect

...

867 2277 14  ---SDG 14---

The text discusses the management of fish stocks and the use of catch quotas in a fishery, which aligns with Sustainable Development Goal 14 (SDG 14) - Life Below Water. SDG 14 aims to conserve and sustainably use the oceans, seas, and marine resources for sustainable development. The text discusses the allocation of catch quotas, the transferability of quotas, and the use of a research-and-development fund, all of which are measures aimed at promoting sustainable fisheries management.
868 42 14  ---SDG 14---
869 775 14  ---SDG 14---
870 3877 14  ---SDG 14---
871 71 14  ---SDG 14--- \\
This text discusses the challenge of integrating emerging ocean industries into existing regulatory structures, which relates to the conservation and sustainable use of the oceans, seas, and marine resources.
872 5815 14  ---SDG 13---

The text discusses the effects of climate change on oceans and the Law of the Sea, as well as the inclusion of oceans in the official negotiatio

911 724 15  ---SDG 15---
912 1167 15  ---SDG 15---
(Note: SDG 15 focuses on protecting, restoring and promoting the sustainable use of terrestrial ecosystems, forests, forests, wetlands, mountains and other ecosystems.)
913 5970 15  ---SDG 15---
914 4268 15  ---SDG 15---
915 6298 15  ---SDG 15---

The text discusses issues related to the measurement and application of biodiversity and ecosystem indicators. The focus on biodiversity and ecosystems aligns with SDG 15, which aims to protect, restore, and promote the sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss. The concerns about establishing causal relations between policies and effects on biodiversity and ecosystems also connect to the goals of SDG 15, as understanding these relationships is crucial for effective conservation and management strategies.
916 588 15  ---SDG 15---
917 2246 15  ---SDG 15---
918 1924 15  ---SDG 15--

946 4121 15  ---SDG 15---

Explanation:

The text provided is about the increase in forest cover and biomass stock in the Nordic countries (Sweden, Norway, Denmark, Iceland, and Finland) and the positive impact it has on the net annual increment of forest growing stock. SDG 15 focuses on protecting, restoring, and promoting the sustainable use of terrestrial ecosystems, including forests, and halting and reversing land degradation and halting biodiversity loss. Therefore, the text can be classified under ---SDG 15---.
947 3264 15  ---SDG 15---

The text discusses the importance of biodiversity mainstreaming, which is the integration of biodiversity considerations into other development agendas. This topic is closely related to SDG 15 (Life on Land), which aims to protect, restore and promote the sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss. The text emphasizes the importance 

971 4998 16  ---SDG 16--- \\
This text discusses the issue of child soldiers in armed conflicts, specifically in Africa, and the disjuncture between the legal obligations and their enforcement at a domestic level. It also mentions the role of international criminal law jurisprudence in enforcing the prohibition. The text highlights the challenges in protecting child soldiers and enforcing laws against their recruitment, which aligns with the focus of SDG 16 on promoting peaceful and inclusive societies, providing access to justice for all, and building effective, accountable, and inclusive institutions.
972 6738 16  ---SDG 16---

Explanation:
The text discusses the jurisprudence of the European Court of Human Rights and the former European Commission of Human Rights in respect of human rights complaints brought under the European Convention on Human Rights (ECHR) about issues relating to the possession, production or distribution of materials classified as pornographic or obscene. The 

997 322 16  ---SDG 16---
998 6351 16  ---SDG 16---

The text discusses the socio-historical aspects of constitutionalism in Argentina, focusing on the political uses of Constitutional Law and the intersection of its agents in the political space. The topic is related to SDG 16, which promotes peaceful and inclusive societies for sustainable development, provides access to justice for all and builds effective, accountable and inclusive institutions at all levels.
999 4375 16  ---SDG 16--- | The text discusses the impact of patent cases on the right to access science and its benefits, which relates to the rule of law and access to justice. |
|---|---|

*The justification for this classification is that the text discusses the implications of patent cases on the right to access science and its benefits, which falls under SDG 16, which promotes peaceful and inclusive societies, access to justice, and effective and accountable institutions. The text highlights the importance of the rule of l

In [17]:
files = glob.glob(output_datapath + "*.pkl")
# files

In [18]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    #print(res)
    try:
        if len(res[3]['choices'])>1:
            raise Exception('ERROR: Multilabel detected!')
        sdg_pred = res[3]['choices'][0]['message']['content']
    except:
        sdg_pred = res
    res4df = (res[0], res[1], res[2], sdg_pred)    
    print(res4df)
    tuple_list.append(res4df)

('1015', 4695, 16, ' ---SDG 16---\n\nExplanation:\nThe text discusses the alliances between human rights NGOs and labor unions in the struggle against military dictatorship in Nigeria. The focus is on the relationships within civil society and how conflicts promote social capital within it. The topic of the text is related to SDG 16, which aims to promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels.')
('582', 714, 9, ' ---SDG 9---')
('2', 6471, 0, ' ---SDG 16---\n\nThe topic of this text is the suspension of a life sentence awarded to a convict in a criminal case in India, and the lack of evidence to support certain claims made against the convict. This is related to SDG 16, which aims to promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all level

In [19]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [20]:
results_processed

,cnt,id,true_label,predicted_label
0,1015,4695,16,---SDG 16---\n\nExplanation:\nThe text discus...
1,582,714,9,---SDG 9---
2,2,6471,0,---SDG 16---\n\nThe topic of this text is the...
3,782,2180,13,---SDG 13---
4,601,4947,10,---SDG 10---
...,...,...,...,...
1015,152,5700,2,---SDG 2---\n\nThe text emphasizes the import...
1016,276,1547,4,---SDG 4---
1017,554,5840,9,---SDG 9---
1018,927,6479,15,---SDG 15---


In [21]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 11--- (Sustainable Cities and Communit...
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 0---\n\nThe text is about a model's ph...
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---\n\nThe topic of this text is the...
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146,3,1786,0,"---SDG 9--- (Industry, Innovation and Infrast..."
4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124,4,2143,0,---SDG 7---
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100,1015,4695,16,---SDG 16---\n\nExplanation:\nThe text discus...
1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60,1016,6628,16,---SDG 16---
1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124,1017,4563,16,---SDG 16---
1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109,1018,6710,16,---SDG 16---


In [22]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,predicted_label_clean
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 11--- (Sustainable Cities and Communit...,11
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 0---\n\nThe text is about a model's ph...,0
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---\n\nThe topic of this text is the...,16
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146,3,1786,0,"---SDG 9--- (Industry, Innovation and Infrast...",9
4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124,4,2143,0,---SDG 7---,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100,1015,4695,16,---SDG 16---\n\nExplanation:\nThe text discus...,16
1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60,1016,6628,16,---SDG 16---,16
1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124,1017,4563,16,---SDG 16---,16
1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109,1018,6710,16,---SDG 16---,16


In [23]:
results_processed_merged.groupby("predicted_label_clean").count()

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,,,
0,34,34,2,34,34,2,2,2,34,34,34,34,34
1,73,73,72,73,73,72,72,72,73,73,73,73,73
10,36,36,36,36,36,36,36,36,36,36,36,36,36
11,73,73,69,73,73,69,69,69,73,73,73,73,73
12,61,61,61,61,61,61,61,61,61,61,61,61,61
13,60,60,60,60,60,60,60,60,60,60,60,60,60
14,58,58,58,58,58,58,58,58,58,58,58,58,58
15,59,59,57,59,59,57,57,57,59,59,59,59,59
16,56,56,54,56,56,54,54,54,56,56,56,56,56


In [24]:
results_processed_merged.to_excel(output_datapath+final_report_name)

In [25]:
results_processed_merged = results_processed_merged[~results_processed_merged.predicted_label_clean.isna()]

In [26]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1018
1018
              precision    recall  f1-score   support

           0       0.94      0.53      0.68        60
           1       0.71      0.87      0.78        60
           2       0.89      0.85      0.87        60
           3       0.80      0.93      0.86        60
           4       0.90      0.95      0.93        60
           5       0.87      0.90      0.89        60
           6       1.00      0.95      0.97        60
           7       0.92      0.90      0.91        60
           8       0.69      0.73      0.71        60
           9       0.74      0.90      0.81        59
          10       0.89      0.54      0.67        59
          11       0.79      0.97      0.87        60
          12       0.92      0.93      0.93        60
          13       0.88      0.88      0.88        60
          14       0.97      0.93      0.95        60
          15       0.92      0.90      0.91        60
          16       0.93      0.87      0.90        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
# AGGIUNTO IL 2024.05.07:

In [28]:
# CORREZIONI NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI O OUTPUT NON UTILI PER DEFINIRE LA CLASSE O CLASSI NON PREVISTE 

import random

# NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI SELEZIONO SEMPRE IL PRIMO
results_processed_merged = results_processed_merged.drop_duplicates(subset='text_id', keep='first')

# # NEL CASO IN CUI IL MODELLO RESTITUISCA OUTPUT NON UTILI PER DEFINIRE LA CLASSE SI APPLICA LA CLASSE 0
# # results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = 0
# results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'sdg'] - 1

# # NEL CASO IN CUI IL MODELLO RESTITUISCA CLASSI NON PREVISTE SI APPLICA LA CLASSE 0
# results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if int(x)>16 else x)

# Lista di valori sdg_list
sdg_list = [str(x) for x in range(17)]
# sdg_list = [x for x in range(17)]
# print(sdg_list)

# Se il valore di 'predicted_label_clean' e' NaN o un valore numerico non presente nella lista sdg_list 
# allora assegna a  'predicted_label_clean' un valore qualunque estratto a caso da sdg_list ma diverso dal valore di 'sdg' per la riga in esame
# Modifica della colonna 'predicted_label_clean' secondo le regole specificate
for index, row in results_processed_merged.iterrows():
    if pd.isna(row['predicted_label_clean']) or row['predicted_label_clean'] not in sdg_list:
#         new_value = random.choice([x for x in sdg_list if x != row['sdg']])
        results_processed_merged.at[index, 'predicted_label_clean'] = 0  # new_value

In [29]:
results_processed_merged.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,predicted_label_clean
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 11--- (Sustainable Cities and Communit...,11
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 0---\n\nThe text is about a model's ph...,0
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---\n\nThe topic of this text is the...,16


In [30]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1018
1018
              precision    recall  f1-score   support

           0       0.84      0.53      0.65        60
           1       0.71      0.87      0.78        60
           2       0.89      0.85      0.87        60
           3       0.80      0.93      0.86        60
           4       0.90      0.95      0.93        60
           5       0.87      0.90      0.89        60
           6       1.00      0.95      0.97        60
           7       0.92      0.90      0.91        60
           8       0.69      0.73      0.71        60
           9       0.74      0.90      0.81        59
          10       0.89      0.54      0.67        59
          11       0.79      0.97      0.87        60
          12       0.92      0.93      0.93        60
          13       0.88      0.88      0.88        60
          14       0.97      0.93      0.95        60
          15       0.92      0.90      0.91        60
          16       0.93      0.87      0.90        60

    accuracy    

In [31]:
classification_report_filename = output_datapath+f'/{experiment_name}_classification_report.xlsx'
classification_report_filename

report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100
report_df.to_excel(classification_report_filename)

In [32]:
report_df

,precision,recall,f1-score,support
0,84.210526,53.333333,65.306122,60.000000
1,71.232877,86.666667,78.195489,60.000000
2,89.473684,85.000000,87.179487,60.000000
3,80.000000,93.333333,86.153846,60.000000
4,90.476190,95.000000,92.682927,60.000000
5,87.096774,90.000000,88.524590,60.000000
6,100.000000,95.000000,97.435897,60.000000
7,91.525424,90.000000,90.756303,60.000000
8,68.750000,73.333333,70.967742,60.000000
9,73.611111,89.830508,80.916031,59.000000


In [33]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [34]:
print(precision, recall, accuracy, f1_score)

86.22284901043899 85.5333998005982 85.55992141453831 85.23099682741012


In [35]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))

86.2 85.5 85.6 85.2
